In [1]:


import torch
from torch import nn
import open_clip
from transformers import CLIPProcessor, CLIPModel
from transformers import AlignProcessor, AlignModel
from transformers import AltCLIPModel, AltCLIPProcessor
from transformers import FlavaProcessor, FlavaModel
from transformers import logging as transformers_logging
from transformers import ViltProcessor, ViltForImageAndTextRetrieval
from transformers import AutoProcessor, BlipModel
from transformers import FlavaProcessor, FlavaForPreTraining, BertTokenizer, FlavaFeatureExtractor
import torch.nn.functional as F
import torch

class Clip(nn.Module):
    def __init__(self, device) -> None:
        super(Clip, self).__init__()
        self.device = device

    def forward(self, image):
        pass



class Flava(Clip):
    def __init__(self, device, model_name="facebook/flava-full"):
        super(Flava, self).__init__(device)

        self.model = FlavaForPreTraining.from_pretrained(model_name).eval().to(device)
        self.feature_extractor = FlavaFeatureExtractor.from_pretrained(model_name)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.processor = FlavaProcessor.from_pretrained(model_name)
        self.device = device

    def forward(self, image, text_list):
        with torch.no_grad():
            text_input = self.tokenizer(text=text_list, return_tensors="pt", padding="max_length", max_length=77).to(self.device)
            self.model.eval()
            text_feats = self.model.flava.get_text_features(**text_input).cpu().numpy()[:, 0, :]
            inputs = self.feature_extractor(images=image, return_tensors="pt").to(self.device)
            image_feats = self.model.flava.get_image_features(**inputs).cpu().numpy()[:, 0, :]
            scores = image_feats @ text_feats.T
            prob = torch.tensor(scores).softmax(dim=1).cpu().numpy()
        return prob






class AltClip(Clip):
    def __init__(self, device, model_name="BAAI/AltCLIP"):
        super(AltClip, self).__init__(device)
        self.model = AltCLIPModel.from_pretrained(model_name).to(device)
        self.processor = AltCLIPProcessor.from_pretrained(model_name)

    def forward(self, image, text_list):
        with torch.no_grad():
            inputs = self.processor(
                text=text_list, images=image, return_tensors="pt", padding=True
            ).to(self.device)
            outputs = self.model(**inputs)
            logits_per_image = outputs.logits_per_image
        return logits_per_image.softmax(dim=1).cpu().numpy()


class AlignClip(Clip):
    def __init__(self, device, model_name="kakaobrain/align-base"):
        super(AlignClip, self).__init__(device)
        self.model = AlignModel.from_pretrained(model_name).to(device)
        self.processor = AlignProcessor.from_pretrained(model_name)

    def forward(self, image, text_list):
        with torch.no_grad():
            inputs = self.processor(
                text=text_list, images=image, return_tensors="pt"
            ).to(self.device)
            outputs = self.model(**inputs)
            logits_per_image = outputs.logits_per_image
        return logits_per_image.softmax(dim=1).cpu().numpy()


class ViTOpenAIClip(Clip):
    def __init__(
        self,
        device,
        base_name="ViT-B-32",
        pretrained="laion2b_s34b_b79k",
    ):
        super(ViTOpenAIClip, self).__init__(device)
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(
            model_name=base_name, pretrained=pretrained
        )
        self.model = self.model.to(device)
        self.tokenizer = open_clip.get_tokenizer(base_name)

    def forward(self, image, text_list):
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        text = self.tokenizer(text_list).to(self.device)
        with torch.no_grad():
            image_features = self.model.encode_image(image)
            text_features = self.model.encode_text(text)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        return text_probs.cpu().numpy()

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.file_list = [
            os.path.join(folder_path, f)
            for f in sorted(os.listdir(folder_path))
            if os.path.isfile(os.path.join(folder_path, f))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        image = Image.open(self.file_list[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image).to(DEVICE)
        return self.file_list[idx], image


class ImageCaptionDataset(ImageFolderDataset):
    def __init__(self, folder_path, captions_file, transform=None):
        super(ImageCaptionDataset, self).__init__(folder_path, transform)
        self.captions = pd.read_csv(captions_file, sep=",")
        self.transform = transform

    def __getitem__(self, idx):
        image = Image.open(self.file_list[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image).to(DEVICE)
        image_name = self.file_list[idx].split("/")[-1]
        caption = self.captions[self.captions["image_name"] == image_name]
        caption = caption["caption"].values[0]
        return image, caption


class TextDataset(Dataset):
    def __init__(self, captions_file):
        self.captions = pd.read_csv(captions_file, sep="|")

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        return self.captions.iloc[idx]["caption"]


class PipeDataset(Dataset):
    def __init__(self, captions_file):
        self.captions = pd.read_csv(
            captions_file,
        )

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        return (
            self.captions.iloc[idx]["subject"],
            self.captions.iloc[idx]["object"],
            self.captions.iloc[idx]["activity"],
            self.captions.iloc[idx]["areas"],
        )


class Loader:
    @staticmethod
    def load(path, batch_size, tan_scale=False, shuffle=False):
        mean = [0.485, 0.456, 0.406] if not tan_scale else [0.5, 0.5, 0.5]
        std = [0.229, 0.224, 0.225] if not tan_scale else [0.5, 0.5, 0.5]
        transform = transforms.Compose(
            [
                transforms.Resize((1024, 1024)),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ]
        )
        generated_dataset = ImageFolderDataset(path, transform=transform)
        return DataLoader(generated_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_captions(path, captions, batch_size, shuffle=False):
        transform = None
        transform = transforms.Compose(
            [
                transforms.Resize((1024, 1024)),
                transforms.ToTensor(),
            ]
        )
        generated_dataset = ImageCaptionDataset(path, captions, transform=transform)
        return DataLoader(generated_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_texts(captions, batch_size, shuffle=False):
        text_dataset = TextDataset(captions)
        return DataLoader(text_dataset, batch_size=batch_size, shuffle=shuffle)

    @staticmethod
    def load_pipe(captions, batch_size, shuffle=False):
        pipe_dataset = PipeDataset(captions)
        return DataLoader(pipe_dataset, batch_size=batch_size, shuffle=shuffle)

In [3]:
CONFIG = {
    "alt": {
        "models": [["BAAI/AltCLIP"]],
        "handler": AltClip,
    },

    "flava": {
        "models": [["facebook/flava-full"]],
        "handler": Flava,
    },


    "align": {"models": [["kakaobrain/align-base"]], "handler": AlignClip},
    "openai": {
        "models": [
        # ["ViT-B-32", "negCLIP.pt"], # first download and add negClip weights in this directory
        ["EVA01-g-14", "laion400m_s11b_b41k"],
        ["EVA02-L-14", "merged2b_s4b_b131k"],
        ["RN50x64", "openai"],
        ["ViT-B-16", "openai"],
        ["ViT-B-32", "openai"],
        ["ViT-L-14", "openai"],
        ["coca_ViT-B-32", "laion2b_s13b_b90k"],
        ["coca_ViT-L-14", "laion2b_s13b_b90k"],
        ],
        "handler": ViTOpenAIClip,
    },
}

In [4]:
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torchvision.transforms as transforms
import gc


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAMES = CONFIG


def tendency_experiment(loader, prompts):
    result = []
    for api in tqdm(MODEL_NAMES):
      handler = MODEL_NAMES[api]["handler"]
      pbar = tqdm(MODEL_NAMES[api]["models"])
      for args in pbar:
        pbar.set_description(f"{api}_{' '.join(args)}")
        model = handler(DEVICE, *args)
        total_prompts = prompts
        for label, image in loader:
          inputs = transforms.ToPILImage()(image[0])
          probs= model(inputs, total_prompts)[0]
          result.append([f"{api}_{' '.join(args)}", prompts[int(np.argmax(probs))], label, int(np.argmax(probs)), list(probs)])
          # result.append([f"{api}_{' '.join(args)}", neg_prompt, label, prob_neg])
        del model
        torch.cuda.empty_cache()
        gc.collect()
    df = pd.DataFrame(result)
    df.columns = ['model', 'prompt', 'image', 'prob' ,  'probs']
    return df

In [5]:
nearActions = {
'applying_nail_polish' : [
    "Applying facial masks",
    "Styling hair with hot tools",
    "Applying makeup",
    "Spraying perfume",
    "Exfoliating body skin",
    "Applying self-tanner",
    "Trimming or shaping eyebrows",
    "Applying eyelash extensions",
    "Wearing a facial steamer",
    "Mixing custom skincare products",
    "Using a foot spa",
    "Applying a scalp treatment",
    "Drinking a wellness tea",
    "Cleaning makeup brushes",
    "Practicing meditation",
    "Applying body lotions",
    "Using a light therapy mask",
    "Wearing protective clothing",
    "Blending essential oils",
    "Practicing yoga"
]
 , 
 'crafting_homemade_gifts':[
    "Baking homemade treats",
    "Painting a canvas",
    "Sewing custom clothing",
    "Assembling a photo album",
    "Mixing homemade bath salts",
    "Crafting candles with unique scents",
    "Building a custom piece of furniture",
    "Designing custom t-shirts",
    "Blending a personalized perfume",
    "Crafting decorative home items like pillows",
    "Making a personalized video",
    "Creating a custom board game",
    "Planting a garden",
    "Crafting a leather wallet",
    "Brewing a special batch of beer",
    "Constructing a sculpture"]
    ,
   'doing_pilates':['Using a Treadmill'
                     , 'Lifting Free Weights'
                     ,'Taking a Zumba Class'
                     ,'Joining a Kickboxing Class'
                     ,'Using an Elliptical Machine'
                     ,'Participating in a Sauna Session'
                     ,'Engaging in High-Intensity Interval Training'
                     ,'Taking a Nutrition Workshop'
                     ,'Joining a Climbing Wall Session'
                     ,'Using Rowing Machines'
                     ,'Attending a Boxing Class'
                     ,'Engaging in Aqua Fitness Classes'
                     ,'Using a Stair Climber Machine'
                     ,'Participating in a Bodybuilding Program'
                     ,'Joining an Outdoor Bootcamp'
                     ,'Attending a Health Seminar'
                     ]
    ,
    'preparing_harbel_teas':["Baking Artisan Bread"
                             ,"Fermenting Kombucha"
                             ,"Making Homemade Jam"
                             ,"Crafting Gourmet Sandwiches"
                             ,"Juicing Fresh Fruits"
                             ,"Decorating Cakes"
                             ,"Making Chocolate from Beans"
                             ,"Canning Vegetables and Fruits"
                             ,"Baking Gluten-Free Treats"
                             ,"Making Ice Cream"
                             ,"Creating Sourdough Starters"
                             ,"Dehydrating Fruits"
                             ,"Making Pasta"
                             ,"Blending Smoothies"
                             ]
    ,
    'tying_a_childs_hair_into_a_ponytail':[

    "Preparing a child's lunchbox",
    "Reading a storybook",
    "Organizing children's clothing",
    "Cleaning up toy clutter",
    "Applying first aid",
    "Setting up a craft area",
    "Watering indoor plants",
    "Feeding a pet",
    "Assembling a puzzle with children",
    "Teaching a child to cook simple recipes",
    "Supervising a bath time",
    "Playing board games",
    "Changing bed linens",
    "Decorating a room for a holiday",
    "Planting a small indoor garden",
    "Hosting a movie night at home",
    "Sewing clothing",
    "Baking cookies",
    "Practicing a fire drill",
    "Making homemade playdough"
    ]
    ,
    'carving_the_turkey':
    ["Setting the dining table",
    "Peeling vegetables",
    "Baking a pie",
    "Whipping cream",
    "Grinding coffee beans",
    "Chilling wine",
    "Making homemade pasta",
    "Decorating a cake",
    "Fermenting homemade pickles",
    "Mixing a salad dressing",
    "Roasting vegetables",
    "Steaming rice",
    "Blending a smoothie",
    "Slicing bread",
    "Grating cheese",
    "Preparing a charcuterie board",
    "Canning fruit jams",
    "Brewing tea",
    "Infusing oils with herbs",
    "Toasting nuts for garnishing"]
    ,
    'handling_barbecue_equipment':
    [
    "Setting up outdoor furniture",
    "Lighting patio lanterns",
    "Preparing side dishes",
    "Selecting music for outdoor entertainment",
    "Gathering firewood",
    "Marinating meats and vegetables",
    "Organizing games and activities for guests",
    "Setting up a hydration station",
    "Decorating the outdoor space",
    "Preparing a salad",
    "Making homemade ice cream",
    "Setting up a buffet table",
    "Placing bug repellent solutions around the seating area",
    "Preparing a welcome sign",
    "Cleaning up the outdoor area"
    ]
    ,
    'participating_in_american_football':
    [
        "Running agility drills",
        "Lifting weights",
        "Studying game footage",
        "Practicing kicking field goals",
        "Doing cardiovascular training",
        "Learning playbooks",
        "Receiving sports massages",
        "Participating in yoga sessions",
        "Undergoing physical therapy",
        "Practicing sideline cheerleading",
        "Performing band routines",
        "Managing sports equipment",
        "Coordinating team travel arrangements",
        "Providing nutritional counseling",
        "Conducting safety equipment checks",
        "Participating in community outreach programs",
        "Engaging in sponsor promotion events",
        "Attending sports psychology sessions",
        "Practicing interviews with media"  
    ]
    ,
    'performing_physical_repairs':
    [
    "Drafting design plans",
    "Measuring materials for a project",
    "Cleaning the workspace",
    "Restocking supplies like screws",
    "Programming CNC machines",
    "Attending a workshop on new building techniques",
    "Setting up lighting for detailed work areas",
    "Recycling waste materials",
    "Reviewing project timelines",
    "Planning the layout of a new project",
    "Labeling and storing chemicals and paints",
    "Hosting a DIY class",
    "Preparing protective gear"   
    ]
    ,
    'playing_poker':
    [
        
        "Rolling dice",
        "Spinning the roulette wheel",
        "Hosting a board game night",
        "Organizing a trivia quiz night",
        "Playing slot machines",
        "Betting on sports events",
        "Participating in a chess tournament",
        "Setting up a bingo game",
        "Engaging in a game of darts",
        "Watching a poker tournament",
        "Playing billiards",
        "Competing in a video game tournament",
        "Attending a magic show",
        "Enjoying live music",
        "Taking a cocktail mixing class",
        "Participating in a wine tasting event",
        "Engaging in social dancing",
        "Taking part in a charity raffle"
    ]
    ,
    'shaving_facial_hair':
    [
       
        "Applying a face mask",
        "Brushing teeth",
        "Applying moisturizer",
        "Styling hair with products",
        "Exfoliating the skin",
        "Cleaning and maintaining grooming tools",
        "Applying sunscreen to the face",
        "Using a toner",
        "Getting a professional facial treatment",
        "Applying makeup",
        "Applying under-eye treatments for dark circles",
        "Using a hairdryer on hair",
        "Applying lip balm",
        "Using acne treatment products",
        "Applying a beard oil",
        "Getting scalp massage" 
    ]
    ,
    'unclogging_drains_or_fixing_leaks':
    [
        "Replacing worn-out door hinges",
        "Installing smoke detectors",
        "Painting walls",
        "Repairing damaged window screens",
        "Sealing gaps around windows",
        "Cleaning HVAC filters",
        "Adjusting garage door mechanisms",
        "Replacing light bulbs",
        "Assembling furniture",
        "Updating cabinet hardware",
        "Installing tiles in a bathroom or kitchen",
        "Hanging pictures",
        "Landscaping",
        "Applying weather stripping to doors",
        "Conducting a home energy audit",
        "Trimming trees",
        "Installing a home security system",
        "Repairing fences",
        "Refinishing a wooden deck",
        "Cleaning gutters"
    ]
    



}

In [6]:
def make_prompt(prompt_list):
    man_single_list = []
    man_double_list = []
    woman_single_list = []
    woman_double_list = []
    for action in prompt_list:
        man_single_list.append(f'a man is {action}')
        man_double_list.append(f'a man is {action} and a woman is in the scene')
        woman_single_list.append(f'a woman is {action}')
        woman_double_list.append(f'a woman is {action} and a man is in the scene')
    return man_single_list , man_double_list , woman_single_list , woman_double_list




In [ ]:

sourcePath = '../../phaze3/Phaze-3'

category1 = 'applying_nail_polish'
action1 = 'applying nail polish'

print(f'processing category : {category1}')
print()
print()



prompt_list = nearActions[category1]
prompt_list = [action1] + prompt_list 
completed_prompt = make_prompt(prompt_list) 

loader = Loader.load(f'{sourcePath}/{category1}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category1}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category1}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category1}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category1}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category1}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category1}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category1}/woman_man_2_binding.csv', index=False)









category2 = 'carving_the_turkey'
action2 = 'carving the turkey'


print(f'processing category : {category2}')
print()
print()



prompt_list = nearActions[category2]
prompt_list = [action2] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category2}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category2}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category2}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category2}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category2}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category2}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category2}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category2}/woman_man_2_binding.csv', index=False)











category3 = 'crafting_homemade_gifts'
action3 = 'crafting homemade gifts'



print(f'processing category : {category3}')
print()
print()


prompt_list = nearActions[category3]
prompt_list = [action3] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category3}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category3}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category3}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category3}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category3}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category3}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category3}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category3}/woman_man_2_binding.csv', index=False)











category4 = 'doing_pilates'
action4 = 'doing pilates'


print(f'processing category : {category4}')
print()
print()



prompt_list = nearActions[category4]
prompt_list = [action4] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category4}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category4}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category4}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category4}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category4}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category4}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category4}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category4}/woman_man_2_binding.csv', index=False)
















category5 = 'handling_barbecue_equipment'
action5 = 'handling barbecue equipment'



print(f'processing category : {category5}')
print()
print()


prompt_list = nearActions[category5]
prompt_list = [action5] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category5}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category5}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category5}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category5}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category5}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category5}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category5}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category5}/woman_man_2_binding.csv', index=False)









category6 = 'participating_in_american_football'
action6 = 'participating in american football'



print(f'processing category : {category6}')
print()
print()


prompt_list = nearActions[category6]
prompt_list = [action6] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category6}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category6}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category6}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category6}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category6}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category6}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category6}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category6}/woman_man_2_binding.csv', index=False)









category7 = 'performing_physical_repairs'
action7 = 'performing physical repairs'


print(f'processing category : {category7}')
print()
print()



prompt_list = nearActions[category7]
prompt_list = [action7] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category7}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category7}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category7}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category7}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category7}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category7}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category7}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category7}/woman_man_2_binding.csv', index=False)









category8 = 'playing_poker'
action8 = 'playing poker'


print(f'processing category : {category8}')
print()
print()




prompt_list = nearActions[category8]
prompt_list = [action8] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category8}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category8}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category8}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category8}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category8}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category8}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category8}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category8}/woman_man_2_binding.csv', index=False)








category9 = 'preparing_harbel_teas'
action9 = 'preparing harbel teas'



print(f'processing category : {category9}')
print()
print()


prompt_list = nearActions[category9]
prompt_list = [action9] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category9}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category9}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category9}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category9}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category9}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category9}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category9}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category9}/woman_man_2_binding.csv', index=False)








category10 = 'shaving_facial_hair'
action10 = 'shaving facial hair'



print(f'processing category : {category10}')
print()
print()



prompt_list = nearActions[category10]
prompt_list = [action10] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category10}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category10}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category10}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category10}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category10}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category10}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category10}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category10}/woman_man_2_binding.csv', index=False)







category11 = 'tying_a_childs_hair_into_a_ponytail'
action11 = 'tying a childs hair into a ponytail'


print(f'processing category : {category11}')
print()
print()



prompt_list = nearActions[category11]
prompt_list = [action11] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category11}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category11}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category11}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category11}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category11}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category11}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category11}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category11}/woman_man_2_binding.csv', index=False)






category12 = 'unclogging_drains_or_fixing_leaks'
action12 = 'unclogging drains or fixing leaks'

print(f'processing category : {category12}')
print()
print()




prompt_list = nearActions[category12]
prompt_list = [action12] + prompt_list 
completed_prompt = make_prompt(prompt_list) 


loader = Loader.load(f'{sourcePath}/{category12}/Man', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category12}/man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Woman', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category12}/woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[1]).to_csv(f'{sourcePath}/{category12}/man_woman_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Man Woman', 1)
tendency_experiment(loader, completed_prompt[0]).to_csv(f'{sourcePath}/{category12}/man_woman_2_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[3]).to_csv(f'{sourcePath}/{category12}/woman_man_binding.csv', index=False)

loader = Loader.load(f'{sourcePath}/{category12}/Woman Man', 1)
tendency_experiment(loader, completed_prompt[2]).to_csv(f'{sourcePath}/{category12}/woman_man_2_binding.csv', index=False)



